In [1]:
#Bibliotecas
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.python.keras.utils import np_utils
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
#Carregando os dados
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [3]:
#Pré-processamento dos dados

#Adaptando os dados ao padrão do TensorFlow
x_reshape = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test_reshape = x_test.reshape(x_test.shape[0], 28, 28, 1)
x_reshape = x_reshape.astype('float32')
x_test_reshape = x_test_reshape.astype('float32')

In [4]:
#Padronizando as escalas (Normalization)
x_reshape /= 255 
x_test_reshape /= 255

In [6]:
#Variaveis Dummy
class_train = np_utils.to_categorical(y_train, 10)
class_test = np_utils.to_categorical(y_test, 10)

In [7]:
#Definindo a estrutura da rede neural
classifier = Sequential()
#camadas de convolução 
classifier.add(Conv2D(32, (3,3), input_shape=(28,28,1), activation='relu'))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size=(2,2)))

classifier.add(Conv2D(32, (3,3), activation='relu'))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(Flatten())

#camadas densas
classifier.add(Dense(units= 128, activation='relu'))
classifier.add(Dropout(0.2)) #zerando alguns valores de entrada para evitar o overfitting

classifier.add(Dense(units= 128, activation='relu'))
classifier.add(Dropout(0.2))

#camada de saida
classifier.add(Dense(units=10, activation='softmax'))

In [8]:
classifier.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [9]:
#Treinando o modelo
classifier.fit(x_reshape, class_train, batch_size=128, epochs=5, validation_data=(x_test_reshape, class_test))

Epoch 1/5
469/469 [==============================] - 44s 89ms/step - loss: 0.2024 - accuracy: 0.9376 - val_loss: 0.1398 - val_accuracy: 0.9570
Epoch 2/5
469/469 [==============================] - 41s 88ms/step - loss: 0.0617 - accuracy: 0.9814 - val_loss: 0.0372 - val_accuracy: 0.9885
Epoch 3/5
469/469 [==============================] - 44s 93ms/step - loss: 0.0430 - accuracy: 0.9872 - val_loss: 0.0408 - val_accuracy: 0.9868
Epoch 4/5
469/469 [==============================] - 40s 85ms/step - loss: 0.0354 - accuracy: 0.9897 - val_loss: 0.0315 - val_accuracy: 0.9899
Epoch 5/5
469/469 [==============================] - 39s 84ms/step - loss: 0.0285 - accuracy: 0.9912 - val_loss: 0.0294 - val_accuracy: 0.9905


In [11]:
precision = classifier.evaluate(x_test_reshape, class_test)

313/313 [==============================] - 2s 8ms/step - loss: 0.0294 - accuracy: 0.9905


In [12]:
#Validação cruzada
from sklearn.model_selection import StratifiedKFold
seed = 5
np.random.seed(seed)
(x,y), (x_test, y_test) = mnist.load_data()
x_reshape = x.reshape(x.shape[0], 28, 28, 1)
x_reshape = x_reshape.astype('float32')
x_reshape /= 255
classe = np_utils.to_categorical(y, 10)

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
results = []

In [13]:
for indice_train, indice_test in kfold.split(x_reshape, np.zeros(shape=(classe.shape[0],1))):
    classifier2 = Sequential()
    classifier2.add(Conv2D(32, (3,3), input_shape=(28,28,1), activation='relu'))
    classifier2.add(MaxPooling2D(pool_size=(2,2)))
    classifier2.add(Flatten())
    
    classifier2.add(Dense(units=128, activation='relu'))
    classifier2.add(Dense(units=10, activation='softmax'))
    
    classifier2.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
    classifier2.fit(x_reshape[indice_train], classe[indice_train], batch_size=128, epochs=5)
    
    precision2 = classifier.evaluate(x_reshape[indice_test], classe[indice_test])
    results.append(precision2[1])

Epoch 1/5
375/375 [==============================] - 16s 41ms/step - loss: 0.2485 - accuracy: 0.9279
Epoch 2/5
375/375 [==============================] - 16s 43ms/step - loss: 0.0783 - accuracy: 0.9772
Epoch 3/5
375/375 [==============================] - 16s 42ms/step - loss: 0.0511 - accuracy: 0.9852
Epoch 4/5
375/375 [==============================] - 16s 42ms/step - loss: 0.0368 - accuracy: 0.9895
Epoch 5/5
375/375 [==============================] - 3s 8ms/step - loss: 0.0133 - accuracy: 0.9962
Epoch 1/5
375/375 [==============================] - 16s 39ms/step - loss: 0.2522 - accuracy: 0.9294
Epoch 2/5
375/375 [==============================] - 16s 42ms/step - loss: 0.0824 - accuracy: 0.9757
Epoch 3/5
375/375 [==============================] - 15s 40ms/step - loss: 0.0557 - accuracy: 0.9829
Epoch 4/5
375/375 [==============================] - 15s 40ms/step - loss: 0.0400 - accuracy: 0.9885
Epoch 5/5
375/375 [==============================] - 3s 7ms/step - loss: 0.0117 - accuracy: 0

In [14]:
results_mean = sum(results)/len(results)
results_mean

0.9963666677474976

### Data Augumentation

In [15]:
generate_train = ImageDataGenerator(rotation_range = 7, 
                                   horizontal_flip = True,
                                   shear_range=0.2,
                                   height_shift_range=0.07,
                                   zoom_range=0.2)
generate_test = ImageDataGenerator()

In [16]:
x = generate_train.flow(x_reshape, class_train, batch_size=128)
y = generate_test.flow(x_test_reshape, class_test, batch_size=128)

In [17]:
classifier.fit_generator(x, steps_per_epoch=60000/128,
                        epochs=5,
                        validation_data=y,
                        validation_steps=10000/128)

Epoch 1/5


C:\Users\l.gomes00\AppData\Local\Temp\ipykernel_11572\3878678674.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  classifier.fit_generator(x, steps_per_epoch=60000/128,


468/468 [==============================] - 48s 100ms/step - loss: 0.2191 - accuracy: 0.9351 - val_loss: 0.0569 - val_accuracy: 0.9825
Epoch 2/5
468/468 [==============================] - 45s 95ms/step - loss: 0.1169 - accuracy: 0.9657 - val_loss: 0.0579 - val_accuracy: 0.9823
Epoch 3/5
468/468 [==============================] - 46s 97ms/step - loss: 0.0978 - accuracy: 0.9706 - val_loss: 0.0478 - val_accuracy: 0.9847
Epoch 4/5
468/468 [==============================] - 45s 96ms/step - loss: 0.0892 - accuracy: 0.9731 - val_loss: 0.0537 - val_accuracy: 0.9830
Epoch 5/5
468/468 [==============================] - 46s 99ms/step - loss: 0.0846 - accuracy: 0.9746 - val_loss: 0.0462 - val_accuracy: 0.9868
